# Heart rate Data Thinning

This jupyter notebook is an excerpt of [this](https://www.kaggle.com/code/carlafgomes/fitbit-data-analysis-in-python#2-Data-Processing)  original notebook Kaggle. The original Kaggle notebook 
uses as input [this](https://www.kaggle.com/datasets/arashnic/fitbit) dataset at Kaggle. Before running this notebook,
you should download that Kaggle dataset and put its contents in the folder

`CausalFitbit/input_data/Fitabase Data 4.12.16-5.12.16`

X=input_data/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv

Y=input_data/Fitabase Data 4.12.16-5.12.16/heartrate_reduced_merged.csv

Basically, what this notebook does is to convert an 
input file X to an output file Y. X gives heartrate 
every 10 secs or so. Y gives hourly averages.

X= contains 2.5M rows, 8.5 MB

Y contains 3332 rows, 2 KB

The reason that I've excerpted this portion is that it takes much more time to run than
all other steps in the original notebook. This notebook stores its
results in a file Y at the end. Then the notebook 
`data_preparation.ipynb` uses Y as input. 
This notebook together with `data_preparation.ipynb` perform the same
actions as the original notebook.



In [1]:
# this makes sure it starts looking for things from the SentenceAx folder down.
import os
import sys
os.chdir('../')
sys.path.insert(0,os.getcwd())
print(os.getcwd())

C:\Users\rrtuc\Desktop\backed-up\python-projects\CausalFitbit


In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import glob

In [3]:
fnames= ["dailyActivity",
"heartrate_seconds",
"sleepDay",
"weightLogInfo"]

def get_csv_path(fname):
 return f"input_data/Fitabase Data 4.12.16-5.12.16/{fname}_merged.csv"

dfs = [pd.read_csv(get_csv_path(fname)) for fname in fnames]

In [4]:
# fnames= ["dailyActivity",
# "heartrate_seconds",
# "sleepDay",
# "weightLogInfo"]

import time
start = time.time()

#dfs[1] has columns: Id, Time, Value

# Convert the 'Time' column to a pandas datetime object
dfs[1]['Time'] = pd.to_datetime(dfs[1]['Time'])

hourly_df = dfs[1].groupby(
    ['Id', pd.Grouper(key='Time', freq='1H')])['Value'].mean().reset_index()
hourly_df.rename(columns={'Value': 'PulseHourlyAverage'}, inplace=True)
# Drop the 'Value' column as it's no longer needed
dfs[1].drop(columns=['Value'], inplace=True)

dfs[1] = pd.merge(
    dfs[1], hourly_df[['Id', 'Time', 'PulseHourlyAverage']], on=['Id', 'Time'])

print(dfs[1].head())
print("Heart Rate df shape:", dfs[1].shape)

end = time.time()
delta = (end - start)/60
print("took %.2f minutes to process" % delta)

           Id                Time  PulseHourlyAverage
0  2022484408 2016-04-12 08:00:00           68.562005
1  2022484408 2016-04-12 09:00:00           66.404700
2  2022484408 2016-04-12 10:00:00          106.716075
3  2022484408 2016-04-12 12:00:00           66.230570
4  2022484408 2016-04-12 13:00:00           83.687500
Heart Rate df shape: (3332, 3)
took 6.01 minutes to process


In [5]:
print(dfs[1].columns)

Index(['Id', 'Time', 'PulseHourlyAverage'], dtype='object')


In [6]:
dfs[1].to_csv(get_csv_path("heartrate_reduced"), index=False)